In [64]:
class SegmentTree():
    def __init__(self,nums):
        n = len(nums)
        self.size = n
        self.tree = [0]*n + nums
        for i in range(n-1, 0, -1):
            self.tree[i] = self.tree[i<<1] + self.tree[i<<1|1]
        
            
    def update(self, i, val):
        i += self.size
        self.tree[i] = val
        while i>1:
            self.tree[i>>1] = self.tree[i] + self.tree[i^1]
            i >>= 1
            
    def sumRange(self, l, r):
        l += self.size
        r += self.size
        ans = 0
        while l<=r:
            if l%2==1:
                ans += self.tree[l]
                l += 1
            l >>= 1
            if r%2==0:
                ans += self.tree[r]
                r -= 1
            r >>= 1
        return ans
                
        
            

In [65]:
t = SegmentTree([2,3,5,8,4])
t.tree

[0, 22, 14, 8, 12, 2, 3, 5, 8, 4]

In [66]:
t.update(2,3)
t.tree

[0, 20, 14, 6, 12, 2, 3, 3, 8, 4]

In [67]:
t.sumRange(1,3)

14

In [22]:
class BinaryIndexedTree():
    def buildTree(self, nums):
        if not nums: return None
        n = len(nums)
        self.size = n
        self.tree = [0]*(n+1)
        
        for i in range(n):
            self.update(i,nums[i])
            
            
    def update(self, i, val):
        diff = val - self.sumRange(i,i)
        i += 1
        while i <= self.size:
            self.tree[i] = self.tree[i] + diff
            i = i + (i & -i)
        
    def cumSum(self, i):
        ans = 0
        i += 1
        while i>0:
            ans += self.tree[i]
            i = i - (i & -i)
        return ans
            
    def sumRange(self, l, r):
        return self.cumSum(r) - self.cumSum(l-1)
    
    


In [23]:
t = BinaryIndexedTree()
t.buildTree([2,3,5,8,4])
t.tree

[0, 2, 5, 5, 18, 4]

In [24]:
t.sumRange(1,3)

16

In [56]:
class SegmentTree():
    def __init__(self, nums):
        n = len(nums)
        self.tree = [0] * (4*n)
        self.lazy = [0] * (4*n)
        self.size = n
        self.buildTree(nums)
        
    def _buildTree(self, nums, treeIndex, low, high):
        if low == high:
            self.tree[treeIndex] = nums[low]
            return
        mid = (low+high)//2
        self._buildTree(nums, 2*treeIndex+1, low, mid)
        self._buildTree(nums, 2*treeIndex+2, mid+1, high)
        self.tree[treeIndex] = self._merge(self.tree[2*treeIndex+1], self.tree[2*treeIndex+2])
        
    def buildTree(self, nums):
        self._buildTree(nums, 0, 0, self.size -1)
        
    def _merge(self, left, right):
        return max([left,right])
        
    def _query(self, treeIndex, low, high, i, j):
        if low > j or high < i: return 0
        if i<=low and j>= high: return self.tree[treeIndex]
        
        mid = (low+high)//2
        if i>mid:
            return self._query(2*treeIndex+2, mid+1, high, i, j)
        elif j<=mid:
            return self._query(2*treeIndex+1, low, mid, i, j)
        
        left = self._query(2*treeIndex+1, low, mid, i, mid)
        right = self._query(2*treeIndex+2, mid+1, high, mid+1, j)
        return self._merge(left,right)
    
    def query(self,i,j):
        return self._query(0, 0, self.size-1, i, j)
    
    def _updateVal(self, treeIndex, low, high, arrIndex, val):
        if low == high: 
            self.tree[treeIndex] = val
            return
        mid = (low+high)//2
        if arrIndex > mid:
            self._updateVal(2*treeIndex+2, mid+1, high, arrIndex, val)
        else:
            self._updateVal(2*treeIndex+1, low, mid, arrIndex, val)
        self.tree[treeIndex] = self._merge(self.tree[2*treeIndex+1], self.tree[2*treeIndex+2])
        
    def updateVal(self, arrIndex, val):
        self._updateVal(0, 0, self.size-1, arrIndex, val)
        
    def _updateLazy(self, treeIndex, low, high, i, j, val):
        if self.lazy[treeIndex]!=0:
            self.tree[treeIndex] += (high-low+1)*self.lazy[treeIndex]
            if low != high:
                self.lazy[2*treeIndex+1] += self.lazy[treeIndex]
                self.lazy[2*treeIndex+2] += self.lazy[treeIndex]
            self.lazy[treeIndex] = 0
        if low > high or low > j or high < i: return
        if i<=low and high<=j:
            self.tree[treeIndex] += (high-low+1)*val
            if low!=high:
                self.lazy[2*treeIndex+1] += val
                self.lazy[2*treeIndex+2] += val
            return
        mid = (low+high)//2
        self._updateLazy(2*treeIndex+1, low, mid, i, j, val)
        self._updateLazy(2*treeIndex+2, mid+1, high, i, j, val)
        self.tree[treeIndex] = self._merge(self.tree[2*treeIndex+1], self.tree[2*treeIndex+2])
    
    def updateLazy(self, i, j, val):
        self._updateLazy(0,0,self.size-1,i,j,val)
        
    def _queryLazy(self, treeIndex, low, high, i, j):
        if low > j or high < i: return 0
        if self.lazy[treeIndex]!=0:
            self.tree[treeIndex] += (high-low+1)*self.lazy[treeIndex]
            if low != high:
                self.lazy[2*treeIndex+1] += self.lazy[treeIndex]
                self.lazy[2*treeIndex+2] += self.lazy[treeIndex]
            self.lazy[treeIndex] = 0
        if i<=low and j>=high: return self.tree[treeIndex]
        mid = (low+high)//2
        if i>mid:
            return self._queryLazy(2*treeIndex+2,mid+1,high,i,j)
        elif j<=mid:
            return self._queryLazy(2*treeIndex+1,low,mid,i,j)
        
        left = self._queryLazy(2*treeIndex+1, low, mid, i, mid)
        right = self._queryLazy(2*treeIndex+2,  mid+1, high, mid+1, j)
        return self._merge(left,right)
    
    def queryLazy(self, i, j):
        return self._queryLazy(0, 0, self.size-1, i,j)
                

In [57]:
nums = [18, 17, 13, 19, 15, 11, 20, 12, 33, 25]
tree = SegmentTree(nums)
tree.query(2, 8)
tree.updateVal(2,16)
tree.query(2, 8)

33

In [58]:
tree.updateLazy(2,4,3)
tree.queryLazy(2,8)

33